In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/Spam_SMS.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X=list(df['Message'])

In [ ]:
y=df['Class']

In [ ]:
y.sample(10)

In [ ]:
y=list(y.replace({'spam':0,'ham':1}))

In [ ]:
y[5]

In [ ]:
# if we have multiple class or for even this one we cam also do
#y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
X_train

In [ ]:
!pip install transformers

In [ ]:
#tokenizer call

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encoding=tokenizer(X_train,truncation=True,padding=True,return_tensors="tf")
test_encoding=tokenizer(X_test,truncation=True,padding=True,return_tensors="tf")

In [ ]:
train_encoding[0]

In [ ]:
import tensorflow as tf

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    y_train
)).batch(8)
test_dataset=tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),
    y_test
)).batch(8)

In [ ]:
model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',from_pt=True)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train model
model.fit(train_dataset, epochs=2, validation_data=test_dataset)

# Evaluate
model.evaluate(test_dataset)

In [ ]:
predictions=model.predict(test_dataset)

In [ ]:
logits = predictions.logits
predicted_classes = tf.argmax(logits, axis=1)

In [ ]:
print(predicted_classes)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,predicted_classes)
cm

In [ ]:
print(accuracy_score(y_test,predicted_classes))

old version in pytorch of transformer implementation

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args=TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weigth_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
with training_args.strategy.scope():
  model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer=TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()
trainer.evaluate(test_dataset)
output=trainer.predict(test_dataset)[1]
